In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from mlvern import Forge

In [2]:
iris = load_iris(as_frame=True)
df = iris.frame

print("Dataset loaded. Sample:")
print(df.head())

X = df.drop(columns="target")
y = df["target"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Dataset loaded. Sample:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [3]:
forge = Forge(project="iris_example")

print("🔍 Inspecting data...")
inspection_report = forge.inspect(df, target="target")
print(inspection_report)

print("📊 Running EDA...")
forge.eda(df)

🔍 Inspecting data...
{'metadata': {'generated_at': '2025-12-27T16:19:20.120449+00:00', 'library': 'mlvern', 'version': '0.1.0'}, 'part_1_profiling': {'dataset_shape': {'rows': 150, 'columns': 5, 'memory_mb': np.float64(0.0058), 'sparsity_percent': np.float64(0.0)}, 'schema': {'sepal length (cm)': {'dtype': 'float64', 'non_null_count': 150}, 'sepal width (cm)': {'dtype': 'float64', 'non_null_count': 150}, 'petal length (cm)': {'dtype': 'float64', 'non_null_count': 150}, 'petal width (cm)': {'dtype': 'float64', 'non_null_count': 150}, 'target': {'dtype': 'int64', 'non_null_count': 150}}, 'missing_values': {'total_missing': 0, 'columns_affected': 0, 'details': {}}, 'duplicates': {'total': 1, 'percentage': 0.67, 'by_features': 2}, 'cardinality': {}, 'numeric_ranges': {'sepal length (cm)': {'min': 4.3, 'max': 7.9, 'mean': 5.843333333333334, 'median': 5.8, 'std': 0.828066127977863, 'q25': 5.1, 'q75': 6.4}, 'sepal width (cm)': {'min': 2.0, 'max': 4.4, 'mean': 3.0573333333333337, 'median': 3.0

In [ ]:
print("🤖 Training model...")
model = RandomForestClassifier(
    n_estimators=10,
    random_state=42
)

trained_model, metrics = forge.train(
    model,
    X_train,
    y_train,
    X_val,
    y_val
)

print("Validation metrics:", metrics)

y_pred = trained_model.predict(X_val)
y_prob = trained_model.predict_proba(X_val)[:, 1] if hasattr(trained_model, "predict_proba") else None

# forge.plot(task="classification", y_true=y_val, y_pred=y_pred,y_prob=y_prob)

print("📦 Committing experiment...")
commit_id = forge.commit(
    message="random forest baseline on iris",
    model=trained_model,
    metrics=metrics,
    params={
        "model": "RandomForestClassifier",
        "n_estimators": 10,
        "random_state": 42
    }
)

print(f"✅ Commit created: {commit_id}")

print("🔁 Checking out model...")
restored_model, restored_metrics, restored_params = forge.checkout(commit_id)

acc = accuracy_score(y_val, restored_model.predict(X_val))
print("Restored model accuracy:", acc)
print("Restored params:", restored_params)
